In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, STL10
from torch.utils.data import DataLoader

In [ ]:
torch.cuda.is_available()

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32)),
    transforms.CenterCrop((32, 32)),
])

cifar10_classes = CIFAR10(root='./data', train=True, download=True).classes

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
from torch import nn

#for 227x227
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        
        self.l1 = nn.Sequential(
            nn.Conv2d(3, 96, 11, stride=4, padding=0), # in, out, kernel_size
            nn.BatchNorm2d(num_features=96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.l2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.l3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.l4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.l5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(256*6*6, 4096), # in, out
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
        )
        self.fc3 = nn.Linear(4096, num_classes)

    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)

        x = x.view(-1, 256*6*6)

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

In [ ]:
from torch import nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        
        self.l1 = nn.Sequential(
            nn.Conv2d(3, 48, 3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=48),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.l2 = nn.Sequential(
            nn.Conv2d(48, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.l3 = nn.Sequential(
            nn.Conv2d(128, 192, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.l4 = nn.Sequential(
            nn.Conv2d(192, 192, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.l5 = nn.Sequential(
            nn.Conv2d(192, 128, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),  # Adjust the input size based on the new feature map size
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
        )
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)

        x = x.view(-1, 128 * 4 * 4)

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x


In [ ]:
alexnet = AlexNet()

In [ ]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimiser = optim.SGD(alexnet.parameters(), momentum=0.9, lr=0.005, weight_decay=0.005)

In [ ]:
import torch

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Move the model to CUDA
alexnet.to(device)

In [ ]:
# Inside the training loop
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU

        optimiser.zero_grad()

        outputs = alexnet(inputs)
        labels = labels.squeeze()

        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()

        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    
    model_path = f'model/alexnet_epoch_{epoch+1}.pth'
    torch.save(alexnet.state_dict(), model_path)
    print(f'Model saved at epoch {epoch+1}.')

print('Finished Training')

In [ ]:
import numpy as np

# Assuming you have already trained your model and saved its weights
# Load the trained model
alexnet = AlexNet()
alexnet.load_state_dict(torch.load('model/alexnet_epoch_2.pth'))
alexnet.eval()

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet.to(device)

# Get a batch from the test loader
test_iter = iter(test_loader)
sample_inputs, sample_labels = next(test_iter)
sample_inputs, sample_labels = sample_inputs.to(device), sample_labels.to(device)

# Perform inference
with torch.no_grad():
    output = alexnet(sample_inputs)

# Convert the output to probabilities using softmax (if your model is for classification)
probabilities = torch.nn.functional.softmax(output, dim=1)

# Get the predicted classes
predicted_classes = torch.argmax(probabilities, dim=1)

predicted_classes = predicted_classes.cpu().numpy()
sample_labels = sample_labels.cpu().numpy()

print(predicted_classes)
print(sample_labels)

# Find the common elements
common_elements = np.equal(predicted_classes, sample_labels)

# Calculate the percentage of common elements
percentage_common = np.sum(common_elements) / len(predicted_classes) * 100

print(f"Percentage of common elements: {percentage_common:.2f}%")
